# Output sensor result

In [ ]:
import os
import pandas as pd
import re

def extract_parts(filename):
    match = re.match(r'(Diz|Val)_(\d+)_T(\d+)', filename)
    if match:
        prefix = match.group(1)
        part1 = int(match.group(2))
        part2 = int(match.group(3))
        # Use a sorting key that puts "Diz" (which is 0) before "Val" (which is 1)
        prefix_order = 0 if prefix == "Diz" else 1
        return (prefix_order, part1, part2)
    return (float('inf'), float('inf'), float('inf'))  # Unmatched files last

In [ ]:
import os
import pandas as pd

# Specify the folder path
folder_path = r"./sensor data/SENSOR REPORT"     # Change this to the folder's path

# List all files in the specified folder
files = os.listdir(folder_path)
files.sort(key=extract_parts)


#features 
avg_stance_time_left_list = []
avg_stance_time_right_list = []
avg_swing_time_left_list = []
avg_swing_time_right_list = []
avg_step_time_left_list = []
avg_step_time_right_list = []
avg_step_length_left_list = []
avg_step_length_right_list = []

avg_stance_time_list = []
avg_swing_time_list = []
avg_double_time_list = []
avg_step_time_list = []
cadence_list = []
avg_velocity_list = []
avg_step_length_list = []

trial_names = []

# avg_stance2swing_left_list = []
# avg_stance2swing_right_list = []
# avg_single2double_list = []

for file_name in files: 
    
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)

################ algorithm 
    stance_time_left = df['Stride Time [ms]']* df['Stance Phase left [%]']/10**5
    stance_time_right = df['Stride Time [ms]']* df['Stance Phase right [%]']/10**5
    swing_time_left = df['Stride Time [ms]']* df['Swing Phase left [%]']/10**5
    swing_time_right = df['Stride Time [ms]']* df['Swing Phase right [%]']/10**5
    step_time_left = df['Step Time left [ms]']/10**3
    step_time_right = df['Step Time right [ms]']/10**3
    step_length_left = df['Step Length left [cm]']/10**2
    step_length_right = df['Step Length right [cm]']/10**2

    stance_time = (stance_time_left + stance_time_right) / 2
    swing_time = (swing_time_left + swing_time_right)/2
    double_time =  df['Stride Time [ms]']* df['Double Stance [%]']/10**5  
    cadence = df['Cadence [step/min]'] 
    step_length = (step_length_left + step_length_right)/2
    step_time = (step_time_left + step_time_right)/2
    # Check for which velocity column exists and convert to m/s
    if 'Velocity [km/h]' in df.columns:
        velocity = df['Velocity [km/h]'] / 3.6  # Convert from km/h to m/s
    elif 'Velocity [mph]' in df.columns:
        velocity = df['Velocity [mph]'] * 0.44704  # Convert from mph to m/s    

    # stance2swing_left = stance_time_left/swing_time_left
    # stance2swing_right = stance_time_right/swing_time_right
    # single2double = swing_time/double_time 
################ append 
    avg_stance_time_left_list.append(stance_time_left)
    avg_stance_time_right_list.append(stance_time_right)
    avg_swing_time_left_list.append(swing_time_left)
    avg_swing_time_right_list.append(swing_time_right)
    avg_stance_time_list.append(stance_time)
    avg_step_time_left_list.append(step_time_left)
    avg_step_time_right_list.append(step_time_right)
    avg_step_length_left_list.append(step_length_left)
    avg_step_length_right_list.append(step_length_right)

    
    avg_swing_time_list.append(swing_time)
    avg_double_time_list.append(double_time)
    cadence_list.append(cadence)
    avg_velocity_list.append(velocity)    
    avg_step_length_list.append(step_length)
    avg_step_time_list.append(step_time)
    trial_names.append(file_name)

    # avg_stance2swing_left_list.append(stance2swing_left)
    # avg_stance2swing_right_list.append(stance2swing_right)
    # avg_single2double_list.append(single2double) 
    
gaitcycletime_dict = {
'trial name': trial_names,
'avg_stancetime': avg_stance_time_list,
'avg_swingtime': avg_swing_time_list,
'avg_doublesupporttime': avg_double_time_list,
'avg_steptime': avg_step_time_list,
'avg_steplength': avg_step_length_list,
'avg_velocity': avg_velocity_list,
'avg_cadence': cadence_list, 
'avg_stancetime_left': avg_stance_time_left_list,
'avg_stancetime_right': avg_stance_time_right_list,
'avg_swingtime_left': avg_swing_time_left_list,
'avg_swingtime_right': avg_swing_time_right_list,
'avg_steptime_left': avg_step_time_left_list,
'avg_steptime_right': avg_step_time_right_list, 
'avg_steplength_left': avg_step_length_left_list, 
'avg_steplength_right': avg_step_length_right_list, 
# 'avg_stance2swing_left': avg_stance2swing_left_list,
# 'avg_stance2swing_right': avg_stance2swing_right_list,
# 'avg_single2double': avg_single2double_list, 
}

# Initialize an empty dictionary to store the flattened sensor data
sensor_cycletime_dict = {}

# Loop over your variables and extract the values
for variable in gaitcycletime_dict:
    if variable == "trial name":
        # If variable is trial names, copy as-is (no iloc needed)
        sensor_cycletime_dict[variable] = gaitcycletime_dict[variable]
    else:
        # Otherwise, assume it's a list of Series — take first value
        sensor_cycletime_dict[variable] = [s.iloc[0] for s in gaitcycletime_dict[variable]]


# Convert to DataFrame
df_results_sensor_dict = pd.DataFrame(sensor_cycletime_dict)

# Save to Excel
df_results_sensor_dict.to_excel(r"./sensor data/sensor result.xlsx", index=False)

print("Excel file 'gait_analysis_results_sensor.xlsx' has been saved successfully.")